In [1]:
# import what is needed
from openeye.oechem import *
from __future__ import print_function
from blues import ncmc
from blues import utils
import parmed
from simtk import unit, openmm
from datetime import datetime
from optparse import OptionParser
import os

In [2]:
def PDB2OEMol(pdbfile):
    '''This function takes in a pdbfile as a string (e.g. 'protein.pdb') and reads it into and OEGraphMol'''
    
    # check if file exists
    if os.path.isfile(pdbfile):
        # read file into an input stream
        ifs = oemolistream(pdbfile)
        # set the format of the input stream to pdb (other wise SMI default)
        ifs.SetFormat(OEFormat_PDB)
        # create OEMol destination
        pdb_OEMol = OEGraphMol()
        # assign input stream to OEMol
        OEReadMolecule(ifs, pdb_OEMol)
        return pdb_OEMol
    else:
        print('PDB filename not found.')

In [3]:
def GetTargetAtoms(molecule, residue_list):
    '''This function takes a OEGraphMol PDB structure and a list of residue numbers and 
        generates a dictionary containing all the non-backbone, heavy atom locations and indicies for those residues.
        Note: The atom indicies start at 0 and are thus -1 from the PDB file indicies'''
    
    # Call this function to find atoms and bonds
    OEFindRingAtomsAndBonds(molecule)
    #OEPerceiveResidues(pdb_OEMol)
    if not OEHasResidues(molecule):
            OEPerceiveResidues(molecule, OEPreserveResInfo_All)
    
    # create and clear dictionary to store atoms that make up residue list
    qry_atoms = {}
    qry_atoms.clear()
    
    # track atom count and backbone atoms
    bbatom_ct = 0
    allatom_ct = 0
    
    print('Searching residues for heavy atoms...')
    # loop through all the atoms in the PDB OEGraphMol structure
    for atom in molecule.GetAtoms():
        # count atoms
        allatom_ct = allatom_ct +1
        # count backbone atoms
        if OEIsBackboneAtom():
            bbatom_ct = bbatom_ct + 1
        # check if the atom is a heavy atom
        if atom.GetAtomicNum() > 1:
            # if heavy, find what residue it is associated with
            myres = OEAtomGetResidue(atom)
            # check if the residue number is amongst the list of residues
            if myres.GetResidueNumber() in residue_list:
                # store the atom location in a query atom dict keyed by its atom index
                qry_atoms.update({atom : atom.GetIdx()})
                print('Found',atom) 
    print('\n')
    print (bbatom_ct)
    print (allatom_ct)
    #return dictionary of residue atoms and indicies
    return qry_atoms

In [4]:
def FindHeavyRotBonds(pdb_OEMol, qry_atoms):
    '''This function takes in an OEGraphMol PDB structure as well as a dictionary of atom locations (keys) 
        and atom indicies.  It loops over the query atoms and identifies the bonds associated with each atom.
        It stores and returns the bond indicies (keys) and the two atom indicies for each bond in a dictionary
        **Note: atom indicies start at 0, so are offset by 1 compared to pdb)'''
    
    # Call this function to find atoms and bonds
    OEFindRingAtomsAndBonds(pdb_OEMol)
    
    # create and clear dictionary to store bond and atom indicies that are rotatable + heavy
    rot_atoms = {}
    rot_atoms.clear()
    
    for atom in qry_atoms:
        for bond in atom.GetBonds():
            # retrieve the begnning and ending atoms
            begatom = bond.GetBgn()
            endatom = bond.GetEnd()
            # if begnnning and ending atoms are not Hydrogen, and the bond is rotatable
            if endatom.GetAtomicNum() >1 and begatom.GetAtomicNum() >1 and OEIsRotor:
                # if the bond has not been added to dictionary already..
                # (as would happen if one of the atom pairs was previously looped over)
                if bond.GetIdx() not in rot_atoms:
                    # print the bond index
                    print('Bond number',bond, 'is rotatable and contains only heavy atoms')
                    # store bond index number (key) and atom indicies in dictionary if not already there
                    rot_atoms.update({bond.GetIdx() : {'AtomIdx_1' : bond.GetBgnIdx(), 'AtomIdx_2': bond.GetEndIdx()}})


    # Return dictionary with bond atom indicies keyed by bond index
    return rot_atoms


In [5]:
def GetRotBonds(pdbfile,residue_list):
    '''This function takes in a PDB filename (as a string) and list of residue numbers.  It returns
        a dictionary of rotatable bonds (containing only heavy atoms), that are keyed by bond index 
        and containing the associated atom indicies
        **Note: The atom indicies start at 0, and are offset by -1 from the PDB file indicies'''
    # read .pdb file into OEGraphMol
    structure = PDB2OEMol(pdbfile)
    print('\nPDB file opened into OEGraphMol\n')
    # Generate dictionary containing locations and indicies of heavy residue atoms
    qry_atoms = GetTargetAtoms(structure, residue_list)
    print('Dictionary of query atoms generated from residue list\n')
    # Identify bonds containing query atoms and return dictionary of indicies
    rot_atoms = FindHeavyRotBonds(structure,qry_atoms)
    print('\nRotable bond and atom index dictionary generated with format: {Bond Index: {Atom1Index:, Atom2Index:}}')
    print('\n')
    print(rot_atoms)

In [6]:
#Test GetRotBonds function using .pdb file and list of residues

# Load parmed files and output as pdb
prmtop = utils.get_data_filename('blues', 'tests/data/eqToluene.prmtop')
inpcrd = utils.get_data_filename('blues', 'tests/data/eqToluene.inpcrd')
# Load these into a ParmEd Structure called `struct`
struct = parmed.load_file(prmtop, xyz=inpcrd)
prot = struct.save('protein.pdb', overwrite = True)

# define a residue list
my_residues = [111]

# call function
my_rotatable_atoms = GetRotBonds('protein.pdb', my_residues)


PDB file opened into OEGraphMol

Searching residues for heavy atoms...
Found 1733 N
Found 1735 C
Found 1737 C
Found 1739 C
Found 1743 C
Found 1747 C
Found 1748 O


22340
22340
Dictionary of query atoms generated from residue list

Bond number 1757 (1748O-1747C) is rotatable and contains only heavy atoms
Bond number 1744 (1735C-1733N) is rotatable and contains only heavy atoms
Bond number 1746 (1737C-1735C) is rotatable and contains only heavy atoms
Bond number 1756 (1747C-1735C) is rotatable and contains only heavy atoms
Bond number 1748 (1739C-1737C) is rotatable and contains only heavy atoms
Bond number 1742 (1733N-1731C) is rotatable and contains only heavy atoms
Bond number 1758 (1749N-1747C) is rotatable and contains only heavy atoms
Bond number 1752 (1743C-1737C) is rotatable and contains only heavy atoms

Rotable bond and atom index dictionary generated with format: {Bond Index: {Atom1Index:, Atom2Index:}}


{1744: {'AtomIdx_2': 1733, 'AtomIdx_1': 1735}, 1746: {'AtomIdx_2': 173